In [ ]:
region = "charlie"  # authentication region, does not limit regions for tenant queries
organization_id = None
tenant_ids = None
services = None
title = None
description = None
ioc_file = None
iocs = None
days = 30
TAEGIS_MAGIC_NOTEBOOK_FILENAME = None

In [ ]:
# validation checks

if not organization_id:
    raise ValueError("'organization_id' must be provided.")

if not iocs and not ioc_file:
    raise ValueError("Either 'ioc_file' or 'iocs' must be provided.")

if days < 1:
    raise ValueError("'days' must be at least 1.")

In [ ]:
%load_ext taegis_magic

import logging
from taegis_magic.core.log import get_module_logger, TRACE_LOG_LEVEL

logger = get_module_logger()
logger.setLevel(logging.INFO)
log = logging.getLogger(__name__)

from pathlib import Path
from textwrap import dedent
from datetime import datetime, timezone

from taegis_magic.core.service import get_service
from taegis_magic.pandas.tenants import lookup_first_environment
from taegis_magic.pandas.ioc import threaded_multi_tenant_ioc_search
from taegis_magic.core.notebook import execute_notebook_pool, NotebookContext

In [ ]:
if not title:
    date = datetime.now(timezone.utc).strftime("%Y-%m-%d")
    title = f"IOC Hunt - {date}"

description = description or ""

In [ ]:
service = get_service(tenant_id=organization_id, environment=region)

In [ ]:
%taegis auth login --use-universal-authentication

In [ ]:
tenants_search_filters = []
if organization_id:
    tenants_search_filters.append(f"--filter-by-tenant-hierarchy {organization_id}")
if tenant_ids:
    for tenant_id in tenant_ids:
        tenants_search_filters.append(f"--filter-by-tenant {tenant_id}")
if services:
    for service in services:
        tenants_search_filters.append(f"--filter-by-service {service}")
tenants_search_filter = " ".join(tenants_search_filters)
tenants_search_filter

In [ ]:
%taegis tenants search --tenant "$organization_id" \
    $tenants_search_filter \
    --region "$region" \
    --assign tenants

In [ ]:
tenants = tenants[tenants["id"] != organization_id]
tenants["first_environment"] = tenants.pipe(lookup_first_environment)
tenants[["id", "name", "first_environment"]]

In [ ]:
for id_, environment in tenants[["id", "first_environment"]].itertuples(index=False):
    print(f"Tenant ID: {id_}, Environment: {environment}")

In [ ]:
if not iocs and ioc_file:
    iocs_file = Path(ioc_file)
    if not iocs_file.exists():
        raise FileNotFoundError(f"IOCs file '{iocs_file}' does not exist.")

    iocs = iocs_file.read_text().splitlines()

    if not iocs:
        raise ValueError(f"IOCs file '{ioc_file}' is empty or contains no valid IOCs.")

iocs

In [ ]:
results = threaded_multi_tenant_ioc_search(
    service=service,
    tenants=tenants,
    iocs=iocs,
    days=days,
)
results

In [ ]:
indicators_template = "\n".join([f"- {ioc}" for ioc in iocs])

Path("null_findings.report.md").write_text(
    dedent(
        f"""
# IoC Hunt Report
                                                  
{description}
                                                  
List of Indicators:
{indicators_template}

## Summary of Findings

No indicators of compromise were found in the last {days} days.
"""
    )
)

In [ ]:
notebook_context = []
for tenant_id, region in tenants[['id', 'first_environment']].itertuples(index=False):
    indicators = results[results['counts_by_tenant.tenant_id'] == tenant_id]
    if not indicators.empty:
        notebook_context.append(
            NotebookContext(
                tenant=tenant_id, 
                region=region,
                parameters={
                    'INDICATORS': indicators.to_dict(orient="records"),
                    'INVESTIGATION_TITLE': title,
                })
        )
    else:
        log.warning(f"WARNING: No results found for tenant {tenant_id}")
        %taegis investigations create \
        --title "$title" \
        --key-findings "null_findings.report.md" \
        --priority LOW \
        --type THREAT_HUNT \
        --status AWAITING_ACTION \
        --assignee-id "@customer" \
        --tenant $tenant_id \
        --region $region

In [ ]:
execute_notebook_pool(
    notebook_context=notebook_context,
    notebook_title=title,
    notebook_path="ioc_hunt_report.ipynb",
)